In [70]:
import pandas as pd
import os.path
import numpy as np
import PIL

In [71]:
stat_df = pd.read_csv('scripts/stat_dict.txt', sep = '\t')
rev_stat_df = pd.read_csv('scripts/rev_stat_dict.txt', sep = '\t')
stat_df.head()

,statistic,cutie,jackknifing,bootstrapping,cooks_d
0,pearson,kpc,jkp,bsp,pointwise_kpc
1,spearman,ksc,jks,bss,pointwise_ksc
2,kendall,kkc,jkk,bsk,pointwise_kkc
3,mine,mine,jkm,bsm,pointwise_mine


In [ ]:
def create_img(dist, subset_df, rev_subset_df, method):
    # define image names
    spacer = np.ones(shape=[480,640,4]) * 255
    if method == 'cooks_d':
        for_img_list = ["true_corr_['cookd']_FP_1.png", "true_corr_['cookd']_TP_1.png",
                        "sample_corr_['cookd']_FP_1.png", "sample_corr_['cookd']_TP_1.png"]
        
        for_dir_path = '../Submissions/CUtIe/data_analysis/' + dist + '_' + subset_df[method] + '1fdr0.05/graphs/'
        for_list = [for_dir_path + x for x in for_img_list] # FP, TP, FP, TP

        # order images
        true_imgs = [for_list[1], for_list[0]]
        sample_imgs = [for_list[3], for_list[2]]
        
        opened_true_imgs = []
        for i in true_imgs:
            try:
                opened_true_imgs.append(np.asarray(PIL.Image.open(i)))
            except:
                opened_true_imgs.append(spacer)
                print i
        
        # true_imgs = [PIL.Image.open(i) for i in true_imgs]
        opened_sample_imgs = []
        for i in sample_imgs:
            try:
                opened_sample_imgs.append(np.asarray(PIL.Image.open(i)))
            except:
                opened_sample_imgs.append(spacer)
                print i
        
        # sample_imgs = [PIL.Image.open(i) for i in sample_imgs]
        
        opened_true_imgs.extend([spacer, spacer])
        opened_sample_imgs.extend([spacer, spacer])
        
        true_imgs_comb = np.hstack(opened_true_imgs)
        sample_imgs_comb = np.hstack(opened_sample_imgs)
    
        
    else:
        for_img_list = ['true_corr_false_corr_FP_1.png', 'true_corr_true_corr_TP_1.png', 
                     'sample_corr_false_corr_FP_1.png', 'sample_corr_true_corr_TP_1.png']

        rev_img_list = ['true_corr_false_corr_TN_1.png', 'true_corr_true_corr_FN_1.png', 
                     'sample_corr_false_corr_TN_1.png', 'sample_corr_true_corr_FN_1.png']

        for_dir_path = '../Submissions/CUtIe/data_analysis/' + dist + '_' + subset_df[method] + '1fdr0.05/graphs/'
        rev_dir_path = '../Submissions/CUtIe/data_analysis/' + dist + '_' + rev_subset_df[method] + '1fdr0.05/graphs/'

        rev_list = [rev_dir_path + x for x in rev_img_list] # TN, FN, TN, FN
        for_list = [for_dir_path + x for x in for_img_list] # FP, TP, FP, TP

        true_imgs = [for_list[1], for_list[0], rev_list[0], rev_list[1]]
        sample_imgs = [for_list[3], for_list[2], rev_list[2], rev_list[3]]
        
        opened_true_imgs = []
        for i in true_imgs:
            try:
                opened_true_imgs.append(np.asarray(PIL.Image.open(i)))
            except:
                opened_true_imgs.append(spacer)
                print i
        
        # true_imgs = [PIL.Image.open(i) for i in true_imgs]
        opened_sample_imgs = []
        for i in sample_imgs:
            try:
                opened_sample_imgs.append(np.asarray(PIL.Image.open(i)))
            except:
                opened_sample_imgs.append(spacer)
                print i
        # sample_imgs = [PIL.Image.open(i) for i in sample_imgs]
        
        true_imgs_comb = np.hstack(opened_true_imgs)
        sample_imgs_comb = np.hstack(opened_sample_imgs)
    
    return true_imgs_comb, sample_imgs_comb

In [ ]:
distributions = ['sim_copula_n50_norm_0_1', 
                 'sim_copula_n50_gamma_1_0_100',
                 'sim_copula_n50_lognorm_3_0',
                 'sim_zero_infl_otu_copula_n50_lognorm_3_0']

# for each statistic
for statistic in stat_df['statistic'].values:
    # statistic = 'pearson'
    subset_df = stat_df.set_index('statistic').loc[statistic,:]
    rev_subset_df = rev_stat_df.set_index('statistic').loc[statistic,:]

    # for each distribution
    for dist in distributions:
        # dist = 'sim_copula_n50_norm_0_1'

        # for each of the 4 metrics
        # gather all four metrics in order TP, FP, TN, FN
        # do for true and do for sample (separate images)

        # cuties
        true_ct_imgs_comb, sample_ct_imgs_comb = create_img(dist, subset_df, rev_subset_df, 'cutie')

        #jk_imgs
        true_jk_imgs_comb, sample_jk_imgs_comb = create_img(dist, subset_df, rev_subset_df, 'jackknifing')

        #bs_imgs
        true_bs_imgs_comb, sample_bs_imgs_comb = create_img(dist, subset_df, rev_subset_df, 'bootstrapping')

        #cd_imgs 
        true_cd_imgs_comb, sample_cd_imgs_comb = create_img(dist, subset_df, rev_subset_df, 'cooks_d')

        # combine images
        true_imgs_comb = np.vstack([true_ct_imgs_comb, true_jk_imgs_comb, true_bs_imgs_comb, true_cd_imgs_comb])
        sample_imgs_comb = np.vstack([sample_ct_imgs_comb, sample_jk_imgs_comb, sample_bs_imgs_comb, sample_cd_imgs_comb])

        # insert spacing of same height but 1/8th the width
        spacer = np.ones(shape=[true_imgs_comb.shape[0], true_imgs_comb.shape[1]/8,4]) * 255

        imgs_comb = np.hstack([true_imgs_comb, spacer, sample_imgs_comb])
        imgs_comb = PIL.Image.fromarray(np.uint8(imgs_comb))
        imgs_comb.save('data_analysis/plot_dist_' + statistic + '_' + dist + '.jpg') 